In [14]:
import gym
from gym import spaces
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from agent_torch.core import Runner, Registry
from agent_torch.core.substep import SubstepObservation, SubstepAction, SubstepTransition
from agent_torch.core.helpers import get_by_path, read_config, read_from_file, grid_network

In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
import argparse

In [2]:
# Define helper functions

def get_var(state, var):
    """
    Retrieves a value from the current state of the model.
    """
    return get_by_path(state, var.split('/')) # Turn 'layer1/bias' to '['layer1', 'bias']

In [4]:
# How does this function work? Let's look at how get_by_path is defined

from functools import reduce
import operator
import torch
from torch import nn


def get_by_path(root, items): # This code feels like it could be easily optimized, am I missing something?
    r"""
    Access a nested object in root by item sequence
    """
    property_obj = reduce(operator.getitem, items, root)

    
    if isinstance(property_obj, nn.ModuleDict): # This bit handles PyTorch objects
        return property_obj
    elif isinstance(property_obj, nn.Module):
        return property_obj()
    else:
        return property_obj

## Example


In [13]:
state = {
    'layer1': {
        'weights': torch.tensor([1.0, 2.0]),
        'bias': torch.tensor([0.5])
    },
    'layer2': {
        'weights': torch.tensor([3.0, 4.0]),
        'bias': torch.tensor([0.1])
    }
}

items = ['layer1', 'weights']

weights = get_by_path(state, items)

print(weights)

property_obj = reduce(operator.getitem, items, state)

print(property_obj)

items = ['layer1']

weights = get_by_path(state, items) # This will return a dictionary

print(weights)

tensor([1., 2.])
tensor([1., 2.])
{'weights': tensor([1., 2.]), 'bias': tensor([0.5000])}


### Explanation of the Model

### Define helper functions

In [14]:
# Helper functions for input retrieval

def get_market_update_input(state, input_variables):
    market_state = get_var(state, input_variables['market_state'])
    agents = get_var(state, input_variables['agents'])
    return market_state, agents

def get_rl_agent_observation_input(state, input_variables):
    market_state = get_var(state, input_variables['market_state'])
    llm_features = get_var(state, input_variables['llm_features'])
    policy_network = get_var(state, input_variables['policy_network'])
    return market_state, llm_features, policy_network

def get_rl_agent_learning_input(state, input_variables):
    hedging_error = get_var(state, input_variables['hedging_error'])
    pnl = get_var(state, input_variables['pnl'])
    policy_network = get_var(state, input_variables['policy_network'])
    return hedging_error, pnl, policy_network

In [ ]:
@Registry.register_substep("a", "observation")
class ObserveMarketData(SubstepObservation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, state):